In [173]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.fake import FakeEmbeddings

import json
import logging
import requests

from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_community.vectorstores import Vectara
from langchain_community.vectorstores.vectara import SummaryConfig, MMRConfig, VectaraQueryConfig

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers.self_query.base import SelfQueryRetriever
from vectaraIntegration import get_knowledge

load_dotenv()

True

In [174]:
VECTARA_CUSTOMER_ID = os.getenv("VECTARA_CUSTOMER_ID")
VECTARA_CORPUS_ID = os.getenv("VECTARA_CORPUS_ID")
VECTARA_API_KEY = os.getenv("VECTARA_API_KEY")

vectorstore = Vectara(
                vectara_customer_id=VECTARA_CUSTOMER_ID,
                vectara_corpus_id=VECTARA_CORPUS_ID,
                vectara_api_key=VECTARA_API_KEY
                )

In [161]:
# vectara = Vectara()
# class VectaraQueryConfig:
# k: int = 10
# lambda_val: float = 0.0
# filter: str = ""
# score_threshold: Optional[float] = None
# n_sentence_context: int = 2
# mmr_config: MMRConfig = field(default_factory=MMRConfig)
# summary_config: SummaryConfig = field(default_factory=SummaryConfig)

query_config = VectaraQueryConfig(
    k=7,  # Retrieve 20 documents instead of the default 10
    # lambda_val=0.5,  # Adjust lexical match parameter
    # filter="doc.category='news' and doc.lang='en'",  # Filter by category and language
    score_threshold=0.7,  # Only include documents with a score above 0.7
    n_sentence_context=2,  # Include 3 sentences before/after the match
    # mmr_config=MMRConfig(diversity_bias=0.8),  # Customize MMR configuration
    summary_config=SummaryConfig(is_enabled=True),  # Set a summary length limit
)

vectara_retriever = vectorstore.vectara_query(
    "explain me about commonwealth bank", 
    config = query_config
    )
# vectara_retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", 
#     search_kwargs={"score_threshold": 0.5, 
#                 "k": 5}
#     )

In [158]:
query_config

VectaraQueryConfig(k=7, lambda_val=0.0, filter='', score_threshold=0.5, n_sentence_context=2, mmr_config=MMRConfig(is_enabled=False, mmr_k=50, diversity_bias=0.3), summary_config=SummaryConfig(is_enabled=True, max_results=7, response_lang='eng', prompt_name='vectara-summary-ext-v1.2.0'))

In [162]:
# docs = vectara_retriever.get_relevant_documents("what is commonwealth bank?")
vectara_retriever

[(Document(page_content="Find out more\n\n\nOur board\n\n\nPaul O'Malley\n\n\nChair\n\nRead more\n\n\nMatt Comyn\n\n\nManaging Director and Chief Executive Officer\n\nRead more\n\n\nLyn Cobley\n\n\nIndependent Non-Executive Director\nRead more\n\n\nJulie Galbo\n\n\nIndependent Non-Executive Director\nRead more\n\n\nPeter Harmer\n\n\nIndependent Non-Executive Director\n\nRead more\n\n\nSimon Moutter\n\n\nIndependent Non-Executive Director\n\nRead more\n\n\nMary Padbury\n\n\nIndependent Non-Executive Director\n\nRead more\n\n\nAnne Templeman-Jones\n\n\nIndependent Non-Executive Director\n\nRead more\n\n\nRob Whitfield AM\n\n\nIndependent Non-Executive Director\n\nRead more\n\n\nCompany Secretaries\n\n\nVicki Clarkson\n\n\nCompany Secretary\nRead more\n\n\nKaren O'Flynn\n\n\nCompany Secretary\nRead more\n\n\nMeet our executive leadership team Matt Comyn\n\n\nManaging Director and Chief Executive Officer\n\nRead more\n\n\nAlan Docherty\n\n\nChief Financial Officer\n\nRead more\n\n\nAndrew 

In [179]:
  
'''
 You can see in the results that the specific sources are referenced in the summary. Vectara issues these citations by putting them in [number] format in the summary text, where number starts from 1 and increases by 1 in each result in the responseList.
'''
knowledge_content, summary = get_knowledge("explain me about commonwealth bank", vectorstore) #-> get documents from vectara Corpu

# def get_knowledge(user_query):
#   response = query(VECTARA_CUSTOMER_ID, 
#             VECTARA_CORPUS_ID,
#             "api.vectara.io", 
#             VECTARA_API_KEY,
#             user_query
#             )

#   response_set = response[0]["responseSet"][0]
#   knowledge_content = ""
#   # knowledge_content += f"{response_set['summary'][0]['text']}\n"
#   # knowledge_content += f"{response_set['summary'][0]['text']}\n\nOTHER INFORMATION:\n"
  
#   for i in range(7):
#     print(response_set['response'][i]['text'])
#     # knowledge_content += f"{response_set['response'][i]['text']}\n"

#   return response

# knowledge = get_knowledge("what is commonwealth bank?")

KeyError: 'summary'

In [170]:
data = [((page_content="...", metadata={'source': 'langchain', 'lang': 'eng', 'offset': '0', 'len': '167', 'speech': 'CommonWealth Our company'}), 0.7730848)]

# Access the first element of the list (assuming you only have one data point)
document, score = data[0]

# Access the metadata dictionary
metadata = document.metadata

# Get the source value
source = metadata.get('source')

# Print the source value
print(source)

NameError: name 'Document' is not defined

In [135]:
knowledge

({'responseSet': [{'response': [{'text': 'Commonwealth Bank was founded under the Commonwealth Bank Act in 1911 and commenced operations in 1912, empowered to conduct both savings and general banking business.',
      'score': 0.8127111,
      'metadata': [{'name': 'source',
        'value': '../Bank Product Data/CommonWealth Our company.txt'},
       {'name': 'lang', 'value': 'eng'},
       {'name': 'offset', 'value': '0'},
       {'name': 'len', 'value': '167'}],
      'documentIndex': 0,
      'corpusKey': {'customerId': 0,
       'corpusId': 2,
       'semantics': 'DEFAULT',
       'dim': [],
       'metadataFilter': '',
       'lexicalInterpolationConfig': None},
      'resultOffset': 0,
      'resultLength': 167},
     {'text': 'Features at\na glance\nThe Commonwealth Private\nBank Account is an exclusive\ninterest bearing transaction\naccount for Commonwealth\nPrivate clients.',
      'score': 0.8101821,
      'metadata': [{'name': 'page', 'value': '3'},
       {'name': 'title',

In [3]:
# loader = TextLoader("../Bank Product Data/CommonWealth Our company.txt")
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# docs = text_splitter.split_documents(documents)

In [153]:
# vectara = Vectara.from_documents(
#     docs,
#     embedding=FakeEmbeddings(size=768),
#     doc_metadata={"speech": "CommonWealth Our company"},
# )

In [11]:
# query = "What did the president say about Ketanji Brown Jackson"
# found_docs = vectara.similarity_search(
#     query, n_sentence_context=0)

In [ ]:
# retriever.get_relevant_documents(query)[0]

In [7]:
# from langchain.memory import ConversationBufferMemory

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [56]:
##################################
question = "explain me about commonwealth bank"
##################################

In [62]:
REPO_ID = "google/gemma-7b"
HF_TOKEN = os.getenv("HF_TOKEN")

llm_base = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    max_new_tokens=2000,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    huggingfacehub_api_token=HF_TOKEN
)

# from transformers import pipeline
# pipe = pipeline("text-generation", model="google/gemma-2b", max_new_tokens=2000)

# vectara = Vectara()
# retriever = SelfQueryRetriever.from_llm(
#     llm_base, vectara, verbose=True
# )#document_content_description, metadata_field_info,
# retriever_vectara = vectara.as_retriever()

# https://python.langchain.com/docs/integrations/providers/vectara/vectara_summary/
# summary_config = {"is_enabled": True, "max_results": 5, "response_lang": "eng"}
# retriever = vectorstore.as_retriever(
#     search_kwargs={"k": 3, "summary_config": summary_config}
# )

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\bcamaster\.cache\huggingface\token
Login successful


In [ ]:
# IF USING PIPELINE
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline=pipe)

In [74]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt1 = PromptTemplate.from_template(
    """
    Use these knowledge to give precise and compact answer : {knowledge}
    the question: {question}
    """
)

prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]

llm = LLMChain(prompt=prompt1, llm=llm_base)
# llm = prompt1 | llm_base

In [163]:
# CHAIN | BUT NO RETRIVER

def get_knowledge_content(vectara, query, threshold=0.5):
    found_docs = vectara.similarity_search_with_score(
        query,
        score_threshold=threshold,
        n_sentence_context = 1
    )
    knowledge_content = ""
    for number, (score, doc) in enumerate(found_docs):
        knowledge_content += f"Document {number}: {found_docs[number][0].page_content}\n"
    return found_docs

knowledge_content = get_knowledge_content(vectorstore, question)

# knowledge_content = get_knowledge(question)
# result = llm.invoke({"question": question, "knowledge" : knowledge_content})
# # print(result["knowledge"])

In [164]:
knowledge_content

[(Document(page_content="Matt Comyn\n\n\nManaging Director and Chief Executive Officer\n\nRead more\n\n\nAlan Docherty\n\n\nChief Financial Officer\n\nRead more\n\n\nAndrew Hinchliff\n\n\nGroup Executive, Institutional Banking and Markets\n\nRead more\n\n\nSian Lewis\n\n\nGroup Executive, Human Resources\n\nRead more\n\n\nMonique Macleod\n\n\nGroup Executive Marketing and Corporate Affairs\nRead more\n\n\nKaren O'Flynn\n\n\nGroup General Counsel and Group Executive, Legal & Group Secretariat\nRead more\n\n\nStuart Munro\n\n\nGroup Executive, Group Strategy\nRead more\n\n\nGavin Munroe\n\n\nGroup Executive, Technology and Group Chief Information Officer\nRead more\n\n\nVittoria Shortt\n\n\nASB Chief Executive\n\nRead more\n\n\nAngus Sullivan\n\n\nGroup Executive, Retail Banking Services\n\nRead more\n\n\nSinead Taylor\n\n\nChief Operations Officer\nRead more\n\n\nMichael Vacy-Lyle\n\n\nGroup Executive, Business Banking\nRead more\n\n\nNigel Williams\n\n\nGroup Chief Risk Officer\n\nRead

In [149]:
knowledge_content

"Document 0: Matt Comyn\n\n\nManaging Director and Chief Executive Officer\n\nRead more\n\n\nAlan Docherty\n\n\nChief Financial Officer\n\nRead more\n\n\nAndrew Hinchliff\n\n\nGroup Executive, Institutional Banking and Markets\n\nRead more\n\n\nSian Lewis\n\n\nGroup Executive, Human Resources\n\nRead more\n\n\nMonique Macleod\n\n\nGroup Executive Marketing and Corporate Affairs\nRead more\n\n\nKaren O'Flynn\n\n\nGroup General Counsel and Group Executive, Legal & Group Secretariat\nRead more\n\n\nStuart Munro\n\n\nGroup Executive, Group Strategy\nRead more\n\n\nGavin Munroe\n\n\nGroup Executive, Technology and Group Chief Information Officer\nRead more\n\n\nVittoria Shortt\n\n\nASB Chief Executive\n\nRead more\n\n\nAngus Sullivan\n\n\nGroup Executive, Retail Banking Services\n\nRead more\n\n\nSinead Taylor\n\n\nChief Operations Officer\nRead more\n\n\nMichael Vacy-Lyle\n\n\nGroup Executive, Business Banking\nRead more\n\n\nNigel Williams\n\n\nGroup Chief Risk Officer\n\nRead more\n\n\nH

In [76]:
print(result['knowledge'])

The Commonwealth Bank of Australia, established in 1911 under the Commonwealth Bank Act, commenced operations in 1912 to conduct savings and general banking business [1]. The bank offers exclusive interest-bearing transaction accounts for Commonwealth Private clients [2]. Commonwealth Bank of Australia takes measures to identify and secure permission for content owned by third parties [3].



In [77]:
print(result['text'])

my answers are:  The Australian Prudential Regulation Authority (APRA) is responsible fro ensuring that banks have adequate capital resources. APRA has a number or powers to ensure that banks meet their obligations including suspending or revoking licences, appointing an administrator if a bank becomes insolvent; and taking over management of a failed institution.[4]


In [73]:
print(result)

The first thing you need is a good understanding. You can find this by reading through your notes or textbook before answering any questions on it; make sure there aren't any gaps where information might be missing! Once we have all our bases covered then comes time for some practice with these types of problems - try using different methods such as substitution (replacing variables) instead if possible because sometimes they may seem easier than others but don't forget about those tricky ones either way though...


In [92]:
# gk pake memory dulu
bot = ConversationalRetrievalChain.from_llm(
    llm, retriever_vectara, verbose=True
)

# RetrievalQAWithSourcesChain.from_chain_type(
#     llm=llm_base, chain_type="stuff", retriever=retriever_vectara,
#     chain_type_kwargs={
#         "prompt": PromptTemplate(
#             template=prompt_template,
#             input_variables=["question"])
#         }
#     )

In [35]:
question = "what is commonwealth?"
result = bot.invoke({"question": question})

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [78]:
chat_history = []
question = "what is commonwealth?"
# result = bot.invoke({"question": question, "chat_history": chat_history}) #validation error
# result = bot({"question": question, "chat_history": chat_history}) #validation error

# result = bot.invoke("what is commonwealth?") # gk bisa karena perlu input keys
# result = bot.invoke({"question": question}) #Missing some input keys: {'chat_history'}

c:\Users\bcamaster\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

All rights reserved. Used with permission. Commonwealth Bank of Australia.

Used with permission. Boardmaker™ is a trademark of Tobii Dynavox. Commonwealth Bank of Australia.

Used with permission. Boardmaker™ is a trademark of Tobii Dynavox. Commonwealth Bank of Australia.

If any of the trustees change, you need
to let us know straight away. The trustees
are solely responsible for all transactions
on the account and ensuring that the
SMSF complies with relevant laws. Commonwealth Direct
            Investment Accounts. If your account is linked to a CommSec
trading account, you authorise us to debit
your account for costs relating to your
dealings in securities through CommSec,
including brokerage and other fees. Unles

ValidationError: 1 validation error for Generation
text
  str type expected (type=type_error.str)

In [ ]:
result["answer"]

NameError: name 'result' is not defined